In [116]:
# 1. csv파일을 load

# 2. csv파일의 '영엄장명' column을 읽어서 네이버 지도검색 url에서 넣어서 검색

# 3. 메뉴테그를 순서대로 읽은 뒤 대표메뉴 3가지와 가격을 추출 후 df에 저장
# column: 'menu_1_name, menu_1_price, menu_2_name, menu_2_price, menu_3_name, menu_3_price'

# 4 df를 csv파일로 저장

In [117]:
import os
from time import sleep
import tqdm

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [118]:
# CSV 파일 로드
df = pd.read_csv('../featureSelector/data/c4_general_cafeteria_chuncheon_CC_opened_SS-HJ_sampling.csv', encoding='utf-8')
print("CSV load complete")

# 결과를 저장할 DataFrame 초기화 column: 'index_cafeteria, cafeteria_name, menu_name, menu_price'
result_df = pd.DataFrame({
    'index_cafeteria': [],
    'cafeteria_name': [],
    'menu_name': [],
    'menu_price': []
})

CSV load complete


In [119]:
# # 테스트용 변수 설정
# i = 33
# print("index 변수 설정 완료 i = ", i)
# KEYWORD_INDEX = df['번호'][i]
# KEYWORD = df['사업장명'][i]
# URL = "https://m.map.kakao.com/actions/searchView?q=" + "춘천 " + KEYWORD

# driver = webdriver.Chrome()  # 웹 드라이버 생성
# driver.get(URL)  # 링크 열기
# sleep(1.5)
# serch_html = driver.page_source  # 검색 페이지의 HTML 가져오기
# serch_html_soup = BeautifulSoup(serch_html, 'lxml')  # BeautifulSoup으로 파싱

# try:
#     sleep(1.5)
#     button = driver.find_element(By.CLASS_NAME, 'tit_g')  # 클래스 이름 'tit_g'를 가진 버튼 찾고 클릭
#     button.click()
    
# except NoSuchElementException: # 검색어가 없는 경우 
#     try:
#         print("해당 요소를 찾을 수 없습니다: .tit_g")
#         driver.quit()  # 웹 드라이버 종료
        
#         URL = "https://m.map.kakao.com/actions/searchView?q=" + KEYWORD

#         driver = webdriver.Chrome()  # 웹 드라이버 생성
#         driver.get(URL)  # 링크 열기
#         sleep(1.5)
#         serch_html = driver.page_source  # 검색 페이지의 HTML 가져오기
#         serch_html_soup = BeautifulSoup(serch_html, 'lxml')  # BeautifulSoup으로 파싱
        
#         sleep(1.5)
#         button = driver.find_element(By.CLASS_NAME, 'tit_g')  # 클래스 이름 'tit_g'를 가진 버튼 찾고 클릭
#         button.click()
#     except NoSuchElementException: # 그래도 검색어가 없는 경우
#         driver.quit()  # 웹 드라이버 종료
#         pass
    
# sleep(1.5)
# menu_html = driver.page_source  # 페이지의 HTML 가져오기
# menu_html_soup = BeautifulSoup(menu_html, 'lxml')  # BeautifulSoup으로 파싱
# sleep(1.5)

# # DataFrame 초기화
# extracted_df = pd.DataFrame({
#     'index_cafeteria': [KEYWORD_INDEX] * 3,
#     'cafeteria_name': [KEYWORD] * 3
# })

# # 'name_menu' 클래스를 가진 모든 요소 찾기
# name_menus = menu_html_soup.find_all(class_='name_menu', limit=3)
# data_name = [menu.get_text() for menu in name_menus]

# # 'price_menu' 클래스를 가진 모든 요소 찾기
# price_menus = menu_html_soup.find_all(class_='price_menu', limit=3)
# data_price = [menu.get_text().replace('가격: ', '').replace('\n', '') for menu in price_menus]

# # 데이터 길이 확인 및 DataFrame 업데이트
# if len(data_name) == len(extracted_df):
#     extracted_df['menu_name'] = data_name
#     print("상위 3개 메뉴 추출 완료")
# else:
#     print("메뉴 데이터의 길이가 맞지 않아 추가하지 않습니다.")

# if len(data_price) == len(extracted_df):
#     extracted_df['menu_price'] = data_price
#     print("메뉴별 가격 추출 완료")
# else:
#     print("가격 데이터의 길이가 맞지 않아 추가하지 않습니다.")
    
# driver.quit()  # 웹 드라이버 종료

# result_df = pd.concat([result_df, extracted_df]) # result_df에 extracted_df의 1~3행을 추가

In [120]:
for i in tqdm.tqdm(range(len(df))):
    print("index 변수 설정 완료 i = ", i)
    KEYWORD_INDEX = df['번호'][i]
    KEYWORD = df['사업장명'][i]
    URL = "https://m.map.kakao.com/actions/searchView?q=" + "춘천 " + KEYWORD

    driver = webdriver.Chrome()  # 웹 드라이버 생성
    driver.get(URL)  # 링크 열기
    sleep(1.5)
    serch_html = driver.page_source  # 검색 페이지의 HTML 가져오기
    serch_html_soup = BeautifulSoup(serch_html, 'lxml')  # BeautifulSoup으로 파싱

    try:
        sleep(1.0)
        button = driver.find_element(By.CLASS_NAME, 'tit_g')  # 클래스 이름 'tit_g'를 가진 버튼 찾고 클릭
        button.click()
        
    except NoSuchElementException: # 검색어가 없는 경우 
        try:
            print("해당 요소를 찾을 수 없습니다: .tit_g")
            driver.quit()  # 웹 드라이버 종료
            
            URL = "https://m.map.kakao.com/actions/searchView?q=" + KEYWORD

            driver = webdriver.Chrome()  # 웹 드라이버 생성
            driver.get(URL)  # 링크 열기
            sleep(1.5)
            serch_html = driver.page_source  # 검색 페이지의 HTML 가져오기
            serch_html_soup = BeautifulSoup(serch_html, 'lxml')  # BeautifulSoup으로 파싱
            
            sleep(1.5)
            button = driver.find_element(By.CLASS_NAME, 'tit_g')  # 클래스 이름 'tit_g'를 가진 버튼 찾고 클릭
            button.click()
        except NoSuchElementException: # 그래도 검색어가 없는 경우
            driver.quit()  # 웹 드라이버 종료
            pass
        
    sleep(1.0)
    menu_html = driver.page_source  # 페이지의 HTML 가져오기
    menu_html_soup = BeautifulSoup(menu_html, 'lxml')  # BeautifulSoup으로 파싱
    sleep(1.0)

    # DataFrame 초기화
    extracted_df = pd.DataFrame({
        'index_cafeteria': [KEYWORD_INDEX] * 3,
        'cafeteria_name': [KEYWORD] * 3
    })

    # 'name_menu' 클래스를 가진 모든 요소 찾기
    name_menus = menu_html_soup.find_all(class_='name_menu', limit=3)
    data_name = [menu.get_text() for menu in name_menus]

    # 'price_menu' 클래스를 가진 모든 요소 찾기
    price_menus = menu_html_soup.find_all(class_='price_menu', limit=3)
    data_price = [menu.get_text().replace('가격: ', '').replace('\n', '') for menu in price_menus]

    # 데이터 길이 확인 및 DataFrame 업데이트
    if len(data_name) == len(extracted_df):
        extracted_df['menu_name'] = data_name
        print("상위 3개 메뉴 추출 완료")
    else:
        print("메뉴 데이터의 길이가 맞지 않아 추가하지 않습니다.")

    if len(data_price) == len(extracted_df):
        extracted_df['menu_price'] = data_price
        print("메뉴별 가격 추출 완료")
    else:
        print("가격 데이터의 길이가 맞지 않아 추가하지 않습니다.")
        
    driver.quit()  # 웹 드라이버 종료

    result_df = pd.concat([result_df, extracted_df]) # result_df에 extracted_df의 1~3행을 추가

  0%|          | 0/10 [00:00<?, ?it/s]

index 변수 설정 완료 i =  0


 10%|█         | 1/10 [00:11<01:39, 11.03s/it]

메뉴 데이터의 길이가 맞지 않아 추가하지 않습니다.
가격 데이터의 길이가 맞지 않아 추가하지 않습니다.
index 변수 설정 완료 i =  1


 20%|██        | 2/10 [00:21<01:24, 10.58s/it]

...

상위 3개 메뉴 추출 완료
가격 데이터의 길이가 맞지 않아 추가하지 않습니다.
index 변수 설정 완료 i =  8


 90%|█████████ | 9/10 [01:26<00:09,  9.27s/it]

상위 3개 메뉴 추출 완료
가격 데이터의 길이가 맞지 않아 추가하지 않습니다.
index 변수 설정 완료 i =  9


100%|██████████| 10/10 [01:35<00:00,  9.51s/it]

메뉴 데이터의 길이가 맞지 않아 추가하지 않습니다.
가격 데이터의 길이가 맞지 않아 추가하지 않습니다.


In [121]:
result_df

,index_cafeteria,cafeteria_name,menu_name,menu_price
0,7502.0,백돈춘천점,NaN,NaN
1,7502.0,백돈춘천점,NaN,NaN
2,7502.0,백돈춘천점,NaN,NaN
0,7519.0,야미마라탕,꿔바로우소,NaN
1,7519.0,야미마라탕,새우볶음밥,NaN
2,7519.0,야미마라탕,마라탕,NaN
0,7523.0,곰소젓갈밥상,NaN,NaN
1,7523.0,곰소젓갈밥상,NaN,NaN
2,7523.0,곰소젓갈밥상,NaN,NaN
0,7524.0,호앤호,NaN,NaN
